1.数据加载与切分

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# --- 我们先重新加载和切分数据，确保所有步骤都在一个代码块里 ---
# ## 1. 数据加载与切分

# 加载数据
file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
train_full = pd.read_csv(file_path)

# 分离特征 (X) 和目标 (y)
X_full = train_full.drop('SalePrice', axis=1)
y_full = train_full['SalePrice']

# --- **就放在这里！** ---
# 在切分之前，就从总的特征集里把Id去掉
X_full_no_id = X_full.drop('Id', axis=1)
# -------------------------

# 然后，用不含Id的特征集去进行切分
X_train, X_val, y_train, y_val = train_test_split(X_full_no_id, y_full, test_size=0.2, random_state=42)

2.对数变换

In [8]:

# 对目标变量y进行对数变换
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

3.特征工程

In [9]:
import numpy as np
import pandas as pd

# 这是一个独立的函数，我们可以反复调用它
def feature_engineer_advanced(df):
    """
    对数据集进行高级特征工程，包括智能填充和特征创造。
    """
    # 为了不修改原始数据，先创建副本
    df_fe = df.copy()

    # --- 1. 智能填充：NaN 代表 "没有" ---
    
    # 填充类别型特征
    for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'MasVnrType']:
        df_fe[col] = df_fe[col].fillna('None')

    # 填充数值型特征
    for col in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
                'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                'MasVnrArea']:
        df_fe[col] = df_fe[col].fillna(0)

    # --- 2. 智能填充：用分组中位数填充 "真缺失" ---

    # LotFrontage 很可能和街区(Neighborhood)有关
    # 我们用每个街区LotFrontage的中位数来填充该街区的缺失值
    df_fe['LotFrontage'] = df_fe.groupby('Neighborhood')['LotFrontage'].transform(
        lambda x: x.fillna(x.median())
    )

    # 对于其他一些"真缺失"，我们暂时还用众数填充，作为保底策略
    for col in ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']:
        df_fe[col] = df_fe[col].fillna(df_fe[col].mode()[0])


    # --- 3. 创造新特征 (和我们上次做的一样) ---

    df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['1stFlrSF'] + df_fe['2ndFlrSF']
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
    df_fe['RemodAge'] = df_fe['YrSold'] - df_fe['YearRemodAdd']
    df_fe['TotalBath'] = df_fe['BsmtFullBath'] + (0.5 * df_fe['BsmtHalfBath']) + \
                         df_fe['FullBath'] + (0.5 * df_fe['HalfBath'])
    
    # --- 4. (可选) 删除一些不再需要的原始列 ---
    # 比如我们已经有了HouseAge，YearBuilt可能就不那么重要了
    # df_fe = df_fe.drop(['YearBuilt', 'YearRemodAdd'], axis=1)

    print("✅ 高级特征工程函数已定义。")
    return df_fe

# --- 现在，我们用这个新函数来处理我们的训练集和验证集 ---
# 假设 X_train 和 X_val 已经存在
X_train_advanced = feature_engineer_advanced(X_train)
X_val_advanced = feature_engineer_advanced(X_val)

print("\n查看一下处理后的数据，确认缺失值是否已被处理：")
# 检查处理后的训练集是否还有缺失值
missing_after = X_train_advanced.isnull().sum().sum()
print(f"处理后，训练集剩余缺失值总数: {missing_after}")

✅ 高级特征工程函数已定义。
✅ 高级特征工程函数已定义。

查看一下处理后的数据，确认缺失值是否已被处理：
处理后，训练集剩余缺失值总数: 0


4.预处理pipeline

In [10]:
# --- ## 4. 数据预处理流水线 (Preprocessing Pipeline) ---

# 首先，我们从经过特征工程的数据中舍去Id列
# （虽然我们之前的函数没用到Id，但在这里再确认一遍是好习惯）
if 'Id' in X_train_advanced.columns:
    X_train_final = X_train_advanced.drop('Id', axis=1)
    X_val_final = X_val_advanced.drop('Id', axis=1)
else:
    X_train_final = X_train_advanced
    X_val_final = X_val_advanced

# 1. 重新定义数值列和类别列的列表
# 因为我们增加了新特征（如TotalSF），所以需要更新列表
categorical_cols = [cname for cname in X_train_final.columns if X_train_final[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_final.columns if X_train_final[cname].dtype in ['int64', 'float64']]

# 2. 创建预处理管道 (和之前一样，但现在作用于新数据)
# 对于数值型特征，我们只保留一个中位数填充器，以防万一有未处理的缺失值
numerical_transformer = SimpleImputer(strategy='median')

# 对于类别型特征，我们只需要进行独热编码
# 因为我们的特征工程函数已经填充了所有类别缺失值
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. 组合成一个总的预处理器
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 4. 用预处理器来处理经过高级特征工程的数据
X_train_processed = preprocessor.fit_transform(X_train_final)
X_val_processed = preprocessor.transform(X_val_final)


# 5. 查看最终处理后的数据形状
print("高级特征工程后，训练集形状:", X_train_final.shape)
print("最终处理后，训练集形状:", X_train_processed.shape)
print("\n高级特征工程后，验证集形状:", X_val_final.shape)
print("最终处理后，验证集形状:", X_val_processed.shape)

print("\n✅ 所有数据准备工作已完成！现在可以进入模型训练了。")

高级特征工程后，训练集形状: (1168, 83)
最终处理后，训练集形状: (1168, 304)

高级特征工程后，验证集形状: (292, 83)
最终处理后，验证集形状: (292, 304)

✅ 所有数据准备工作已完成！现在可以进入模型训练了。


5.1.a 使用random forest

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 再次训练模型，但这次用的是经过高级特征工程的数据 ---

# 我们可以叫它 model_v2，以区别于之前的基准模型
model_v2 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

print("正在使用加强版特征数据训练新模型...")
# ‼️注意：我们用的是 X_train_processed 和 y_train_log
model_v2.fit(X_train_processed, y_train_log)
print("✅ 新模型训练完成！")

# 在验证集上进行预测
val_preds_log_v2 = model_v2.predict(X_val_processed)

# 将预测值逆变换回原始尺度
val_preds_v2 = np.expm1(val_preds_log_v2)

# 计算并打印新的评估分数
rmsle_v2 = np.sqrt(mean_squared_log_error(y_val, val_preds_v2))

print("-" * 50)
print(f"我们之前的基准分数为: 0.14561")
print(f"🎉 使用高级特征工程后，新分数为: {rmsle_v2:.5f}")
print("-" * 50)

正在使用加强版特征数据训练新模型...
✅ 新模型训练完成！
--------------------------------------------------
我们之前的基准分数为: 0.14561
🎉 使用高级特征工程后，新分数为: 0.14544
--------------------------------------------------


5.1.b 使用lightGBM

In [12]:
# LightGBM 是一个需要单独安装的库
# 在Kaggle环境中，通常已经预装好了
# 如果在本地，你可能需要运行 !pip install lightgbm
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 使用相同的“加强版”数据，但更换模型 ---

# 1. 定义 LightGBM 回归器模型
# 同样设置 random_state 以保证结果可复现
model_lgbm = lgb.LGBMRegressor(random_state=42)

print("正在使用加强版特征数据训练 LightGBM 模型...")
# ‼️注意：我们用的仍然是 X_train_processed 和 y_train_log
model_lgbm.fit(X_train_processed, y_train_log)
print("✅ LightGBM 模型训练完成！")

# 2. 在验证集上进行预测
val_preds_log_lgbm = model_lgbm.predict(X_val_processed)

# 3. 将预测值逆变换回原始尺度
val_preds_lgbm = np.expm1(val_preds_log_lgbm)

# 4. 计算并打印新的评估分数
rmsle_lgbm = np.sqrt(mean_squared_log_error(y_val, val_preds_lgbm))

print("-" * 50)
print(f"随机森林模型的分数是: 0.14544")
print(f"🚀 使用 LightGBM 模型后，新分数为: {rmsle_lgbm:.5f}")
print("-" * 50)

正在使用加强版特征数据训练 LightGBM 模型...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3653
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 197
[LightGBM] [Info] Start training from score 12.030658
✅ LightGBM 模型训练完成！
--------------------------------------------------
随机森林模型的分数是: 0.14544
🚀 使用 LightGBM 模型后，新分数为: 0.13702
--------------------------------------------------


5.1.c 使用XGBoost

In [15]:
# XGBoost 也是一个需要单独安装的库
# 在Kaggle环境中，通常已经预装好了
# 如果在本地，你可能需要运行 !pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 使用相同的“加强版”数据，但更换为XGBoost模型 ---

# 1. 定义 XGBoost 回归器模型
# 我们给它更多的树(n_estimators=1000)，但用更小的学习率(learning_rate=0.05)
# 同时设置了 early_stopping_rounds，让它在验证集上性能不再提升时自动停止训练
model_xgb = xgb.XGBRegressor(n_estimators=1000, 
                             learning_rate=0.05, 
                             random_state=42, 
                             n_jobs=-1,
                             early_stopping_rounds=5) # 核心：如果验证集分数连续5轮没有提升，就停止训练

print("正在使用加强版特征数据训练 XGBoost 模型...")

# 2. 训练模型，并加入提前停止的设置
# ‼️注意：为了使用early_stopping_rounds，我们需要在.fit()中提供验证集数据
model_xgb.fit(X_train_processed, y_train_log, 
              eval_set=[(X_val_processed, y_val_log)], 
              verbose=False) # verbose=False让它在训练中不打印过多信息

print("✅ XGBoost 模型训练完成！")

# 3. 在验证集上进行预测
val_preds_log_xgb = model_xgb.predict(X_val_processed)

# 4. 将预测值逆变换回原始尺度
val_preds_xgb = np.expm1(val_preds_log_xgb)

# 5. 计算并打印新的评估分数
# 这里需要处理一个边界情况：如果模型预测出负数（虽然罕见），会导致计算错误
# 我们确保所有预测值至少为0
val_preds_xgb[val_preds_xgb < 0] = 0
rmsle_xgb = np.sqrt(mean_squared_log_error(y_val, val_preds_xgb))

print("-" * 50)
print(f"LightGBM 模型的分数是: 0.13702")
print(f"🚀 使用 XGBoost 模型后，新分数为: {rmsle_xgb:.5f}")
print("-" * 50)

正在使用加强版特征数据训练 XGBoost 模型...
✅ XGBoost 模型训练完成！
--------------------------------------------------
LightGBM 模型的分数是: 0.13702
🚀 使用 XGBoost 模型后，新分数为: 0.13893
--------------------------------------------------


5.2 模型融合

In [16]:
# 我们已经有了上一轮的预测结果：
# val_preds_lgbm (LightGBM的预测)
# val_preds_xgb (XGBoost的预测)

# 1. 将两个模型的预测结果做简单的平均
blended_preds = 0.5 * val_preds_lgbm + 0.5 * val_preds_xgb

# 2. 计算融合后的分数
rmsle_blended = np.sqrt(mean_squared_log_error(y_val, blended_preds))

print("-" * 50)
print(f"LightGBM 单模型分数: 0.13702")
print(f"XGBoost 单模型分数:  0.13893")
print(f"🤝 两个模型融合后，最终分数为: {rmsle_blended:.5f}")
print("-" * 50)

--------------------------------------------------
LightGBM 单模型分数: 0.13702
XGBoost 单模型分数:  0.13893
🤝 两个模型融合后，最终分数为: 0.13590
--------------------------------------------------


6.最后生成预测结果

In [6]:
# 假设 preprocessor 和 model 变量已经存在于之前的代码块中

# 1. 加载官方测试数据
test_data_path = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(test_data_path)

# 2. 准备测试数据 (保存Id，并去除Id列)
test_ids = test_df['Id']
test_X = test_df.drop('Id', axis=1)

# 3. 使用【已经拟合过】的预处理器来转换测试数据
# ‼️注意：这里只用 .transform()，绝不能用 .fit_transform()
test_X_processed = preprocessor.transform(test_X)

# 4. 使用【已经训练好】的模型进行预测
test_preds_log = model.predict(test_X_processed)

# 5. 将预测结果逆变换回原始尺度
test_preds = np.expm1(test_preds_log)

# 6. 创建提交文件
submission_df = pd.DataFrame({'Id': test_ids, 'SalePrice': test_preds})

# 7. 保存为 .csv 文件
submission_df.to_csv('submission_baseline.csv', index=False)

print("-" * 50)
print("🎉 提交文件 'submission_baseline.csv' 已成功生成！")
print("现在你可以去Kaggle的 'Output' 部分找到它并提交了。")
print("-" * 50)

--------------------------------------------------
🎉 提交文件 'submission_baseline.csv' 已成功生成！
现在你可以去Kaggle的 'Output' 部分找到它并提交了。
--------------------------------------------------
